## 2. Die gesammelten Daten analysieren


Wir haben bereits viele Daten gesammelt, kennen jedoch bislang nur wenige ihrer Eigenschaften.

Unser Ziel ist es, einen Überblick über die tatsächlich in diesen Daten enthaltenen Informationen zu gewinnen.


Vorab müssen wieder die benötigten Bibliotheken importiert und Konfigurationen gesetzt werden.


In [ ]:
import json
from collections import defaultdict


mode = 1
# 1 = Nationalratswahlen

filename = "nationalrat_raw.json"

Im Folgenden definieren wir eine _Funktion_.
Diese Funktion erlaubt es uns, Daten schöner auszugeben.


In [ ]:
def print_dict(d: dict):
    maxlen_keys = max(map(len, map(str, d.keys())))
    maxlen_values = max(map(len, map(str, d.values())))
    for key in d.keys():
        print(f"{key:.<{maxlen_keys}}{d[key]:.>{maxlen_values+1}}")

Dann folgt das Laden der Daten.


In [ ]:
candidates = json.load(open(filename, encoding="utf-8"))

Wie viele Personen kandidieren für jede Partei?


In [ ]:
party_count = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

# Sortieren nach Anzahl Kandidierende pro Partei
party_count = dict(sorted(party_count.items(), key=lambda item: item[1], reverse=True))
print_dict(party_count)

Wie sieht's aus mit dem Frauenanteil der Kandidierenden?


In [ ]:
# Ein Dictionary für Frauen und Männer, eins nur für Frauen. Kandidierende ohne Eintrag beim Feld "gender" werden ignoriert
all_candidates = defaultdict(int) # Objekt für Frauen und Männer
female_only = defaultdict(int) # Objekt nur für Frauen

# Iteriert über alle Kandidierenden und zählt Frauen und Männer pro Partei
for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    gender = candidate.get("gender")
    # Wenn die Kandidierende Person einer Partei zugeordnet ist
    if party: # and party_count[party] >= 10: # Nur Parteien mit mindestens 10 Kandidierenden
        # Wenn die Kandidierende Person weiblich ist
        if gender == "f":
            female_only[party] += 1
            all_candidates[party] += 1
        # Wenn die Kandidierende Person männlich ist
        if gender == "m":
            all_candidates[party] += 1

# Berechnet den Frauenanteil
age_share = {}
for party_name, young in female_only.items():
    ratio = young / all_candidates[party_name]
    age_share[party_name] = f"{ratio:.0%}"
age_share = dict(sorted(age_share.items(), key=lambda item: int(item[1].strip('%')), reverse=True))

print_dict(age_share)

Dasselbe können wir auch mit dem Alter machen. Im Folgenden analysieren wir, wie die Altersverteilung ausschaut.

Wir wollen den Anteil an Kandidierenden berechnen, die Jünger als 40 (definierbar) sind.


In [ ]:
age_cutoff = 40

# Ein Dictionary für Frauen und Männer, eins nur für Frauen. Kandidierende ohne Eintrag beim Feld "gender" werden ignoriert
all_candidates = defaultdict(int) # Objekt für Frauen und Männer
younger_only = defaultdict(int) # Objekt nur für Frauen

# Iteriert über alle Kandidierenden und zählt Frauen und Männer pro Partei
for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    age = 2024 - candidate.get("yearOfBirth")
    # Wenn die Kandidierende Person einer Partei zugeordnet ist
    if party:
        # Wenn die Kandidierende Person jünger ist
        if age <= age_cutoff:
            younger_only[party] += 1
            all_candidates[party] += 1
        # Wenn die Kandidierende Person älter ist
        else:
            all_candidates[party] += 1

# Berechnet den Altersanteil
age_share = {}
for party_name, young in younger_only.items():
    ratio = young / all_candidates[party_name]
    age_share[party_name] = f"{ratio:.0%}"
age_share = dict(sorted(age_share.items(), key=lambda item: int(item[1].strip('%')), reverse=True))

print_dict(age_share)